In [1]:
import pandas as pd
features = pd.read_csv('feature_MFCCHurstQRDb.csv')
features.head(5)

,Label,1,2,3,4,5,6,7,8,9,...,52,53,54,55,56,57,58,59,60,61
0,0.0,-533.746948,110.161835,-19.112837,-18.278614,6.415710,19.949106,49.810707,-19.014505,5.169472,...,1.497159,1.497159,1.497159,1.497159,1.497159,1.497159,1.497159,1.497159,1.497159,1.497159
1,1.0,-542.197815,114.544655,26.379042,-20.956390,-26.116196,-17.675894,40.142223,7.567637,12.492562,...,1.485870,1.485870,1.485870,1.485870,1.485870,1.485870,1.485870,1.485870,1.485870,1.485870
2,0.0,-523.926819,129.726059,-14.086971,-2.957496,15.298673,9.232695,53.491428,-18.275524,-8.059939,...,1.543358,1.543358,1.543358,1.543358,1.543358,1.543358,1.543358,1.543358,1.543358,1.543358
3,0.0,-463.753571,94.143036,-42.200047,-11.960206,23.792288,15.159955,29.874699,7.889102,-5.222220,...,1.505574,1.505574,1.505574,1.505574,1.505574,1.505574,1.505574,1.505574,1.505574,1.505574
4,1.0,-452.050171,90.309608,39.166100,-24.664038,-4.774487,1.477332,33.049057,0.046250,-8.935451,...,1.459801,1.459801,1.459801,1.459801,1.459801,1.459801,1.459801,1.459801,1.459801,1.459801


In [2]:
features.shape

(327, 62)

In [3]:
# Use numpy to convert to arrays
import numpy as np
# Labels are the values we want to predict
labels = np.array(features['Label'])
# Remove the labels from the features
# axis 1 refers to the columns
features = features.drop('Label', axis = 1)
# Convert to numpy array
features = np.array(features)

### PCA 主成分分析

In [4]:
# # 使用Scikit-learn进行主成分分析
# from sklearn.decomposition import PCA
 
# pca = PCA()
# features_pca = pca.fit_transform(features)

# import matplotlib.pyplot as plt
# # pca.explained_variance_ratio_
# plt.bar(range(1, 62), pca.explained_variance_ratio_, alpha=0.5, align='center')
# plt.step(range(1, 62), np.cumsum(pca.explained_variance_ratio_), where='mid')
# plt.ylabel('Explained variance ratio')
# plt.xlabel('Principal components')
# plt.show()

In [5]:
# pca = PCA(n_components=13)
# features = pca.fit_transform(features)

## SVM手动调参，经过测试，表现最好的kernel是poly

In [6]:
from sklearn.svm import SVC
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [7]:
total_folds = 10
kfold = KFold(n_splits= total_folds, shuffle=True)

acc_per_fold = []
fold=0

for train_index, test_index in kfold.split(features, labels):
    #十折交叉验证，取出训练集和测试集
    fold+=1
    print("\nFold : ",fold,"\n")
    x_train, x_test, y_train, y_test =  features[train_index], features[test_index], labels[train_index], labels[test_index]

    #SVM需要特征标准化
    scaler = StandardScaler()
    scaler.fit(x_train)
    x_train = scaler.transform(x_train)
    x_test = scaler.transform(x_test)
    
    #测试最好的多项式次数，防止欠拟合, 最高20次， 防止过拟合
    best_acc=0
    for degree in range(1,21):
        svc_clf = SVC(C= 10, degree = degree, kernel = 'poly')
        svc_clf.fit(x_train, y_train)
        accuracy = svc_clf.score(x_test, y_test)
        if accuracy > best_acc:
            best_acc = accuracy
            best_degree = degree
            print("New optimal parameters : degree : ", best_degree, ", accuracy : ", accuracy)

    #找到最好的次数，训练
    svc_clf = SVC(C = 10, degree = best_degree, kernel = 'poly')
    svc_clf.fit(x_train, y_train)

    acc_per_fold.append(svc_clf.score(x_test, y_test))

for i in range(total_folds):
    print("Fold ", i, "accuracy", acc_per_fold[i])

print("\nMean Accuracy = ",np.mean(acc_per_fold),'(±',np.std(acc_per_fold),')');


Fold :  1 

New optimal parameters : degree :  1 , accuracy :  0.8181818181818182

Fold :  2 

New optimal parameters : degree :  1 , accuracy :  0.7575757575757576
New optimal parameters : degree :  3 , accuracy :  0.8181818181818182

Fold :  3 

New optimal parameters : degree :  1 , accuracy :  0.8484848484848485

Fold :  4 

New optimal parameters : degree :  1 , accuracy :  0.8787878787878788

Fold :  5 

New optimal parameters : degree :  1 , accuracy :  0.8484848484848485

Fold :  6 

New optimal parameters : degree :  1 , accuracy :  0.9090909090909091

Fold :  7 

New optimal parameters : degree :  1 , accuracy :  0.8484848484848485

Fold :  8 

New optimal parameters : degree :  1 , accuracy :  0.78125

Fold :  9 

New optimal parameters : degree :  1 , accuracy :  0.84375

Fold :  10 

New optimal parameters : degree :  1 , accuracy :  0.75
New optimal parameters : degree :  3 , accuracy :  0.84375
Fold  0 accuracy 0.8181818181818182
Fold  1 accuracy 0.8181818181818182
Fold